## 数据处理

参考文档：[process_data](https://github.com/yerfor/GeneFacePlusPlus/blob/main/docs/process_data/guide-zh.md)

In [ ]:
ffmpeg -i data/raw/videos/wzx.mp4 -vf fps=25,scale=w=512:h=512 -qmin 1 -q:v 1 data/raw/videos/wzx_512.mp4
mv data/raw/videos/wzx.mp4 data/raw/videos/wzx_to_rm.mp4
mv data/raw/videos/wzx_512.mp4 data/raw/videos/wzx.mp4

In [ ]:
export CUDA_VISIBLE_DEVICES=0
export PYTHONPATH=./
export VIDEO_ID=wzx
mkdir -p data/processed/videos/wzx
ffmpeg -i data/raw/videos/wzx.mp4 -f wav -ar 16000 data/processed/videos/wzx/aud.wav 
python data_gen/utils/process_audio/extract_hubert.py --video_id=wzx
python data_gen/utils/process_audio/extract_mel_f0.py --video_id=wzx

In [ ]:
export PYTHONPATH=./
export VIDEO_ID=wzx
export CUDA_VISIBLE_DEVICES=0
mkdir -p data/processed/videos/wzx/gt_imgs
ffmpeg -i data/raw/videos/wzx.mp4 -vf fps=25,scale=w=512:h=512 -qmin 1 -q:v 1 -start_number 0 data/processed/videos/wzx/gt_imgs/%08d.jpg
python data_gen/utils/process_video/extract_segment_imgs.py --ds_name=nerf --vid_dir=data/raw/videos/wzx.mp4 # extract image, segmap, and background

In [ ]:
export PYTHONPATH=./
export VIDEO_ID=wzx
python data_gen/utils/process_video/extract_lm2d.py --ds_name=nerf --vid_dir=data/raw/videos/wzx.mp4

In [ ]:
export PYTHONPATH=./
export VIDEO_ID=wzx
export CUDA_VISIBLE_DEVICES=0
python data_gen/utils/process_video/fit_3dmm_landmark.py --ds_name=nerf --vid_dir=data/raw/videos/wzx.mp4 --reset  --debug --id_mode=global

In [ ]:
export PYTHONPATH=./
export VIDEO_ID=wzx
python data_gen/runs/binarizer_nerf.py --video_id=wzx

## 开始训练

参考文档：[train_and_infer](https://github.com/yerfor/GeneFacePlusPlus/blob/main/docs/train_and_infer/guide-zh.md)

In [ ]:
# 训练 Head NeRF 模型
# 模型与tensorboard会被保存在 `checkpoints/<exp_name>`
python tasks/run.py --config=egs/datasets/wzx/lm3d_radnerf_sr.yaml --exp_name=motion2video_nerf/wzx_head --reset

In [ ]:
# 训练 Torso NeRF 模型
python tasks/run.py --config=egs/datasets/wzx/lm3d_radnerf_torso_sr.yaml --exp_name=motion2video_nerf/wzx_torso --hparams=head_model_dir=checkpoints/motion2video_nerf/wzx_head --reset

In [ ]:
# 推理
python inference/genefacepp_infer.py --head_ckpt= --torso_ckpt=checkpoints/motion2video_nerf/wzx_torso --drv_aud=data/raw/val_wavs/graphics.wav